In [56]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import time

import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import pos_tag, word_tokenize

from gensim.summarization.summarizer import summarize
from gensim import corpora
from gensim.models import LdaModel, LdaMulticore
from gensim.test.utils import datapath

from GoogleNews import GoogleNews
googlenews = GoogleNews()

import newspaper
from newspaper import Article
from newspaper import fulltext


## Блок выгрузки новостей

In [61]:
#Блок выгрузки линков и заголовков новостей из гугла
#новости выгружаются по заданной теме, глубина выгрузки - 10 страниц

TOPIC = 'China'
gnews_links = []
gnews=[]
googlenews.search(TOPIC)

#замеряем время
start_time = time.time()

for i in range(1,3):
    googlenews.clear()
    googlenews.getpage(i)
    for j in range(0, len(googlenews.gettext())):
        gnews.append(googlenews.gettext()[j])
        gnews_links.append(googlenews.getlinks()[j])

print("--- На формирование массива затрачено %s секунд ---" % (time.time() - start_time))             
print('---Выгружено %s статей---' % len(gnews_links))    

--- На формирование массива затрачено 4.9833104610443115 секунд ---
---Выгружено 20 статей---


In [62]:
#блок выгрузки новостей и формирования массива текстов
body = []
count = 0
error_position = [] #массив с номерами битых ссылок

#замеряем время
start_time = time.time()

for url in gnews_links:
    try:
        html = requests.get(url).text
        text = fulltext(html)
        body.append(text)
    except:
        error_position.append(count) #иногда попадаются битые ссылки. Здесь мы будем сохранять их позиции
        pass
    count += 1
    
print("--- На выгрузку затрачено %s секунд ---" % (time.time() - start_time))             


--- На выгрузку затрачено 15.410950422286987 секунд ---


In [63]:
#Блок предобработки текста

stop_words = stopwords.words('english')
stop_words = stop_words + ['china', 'chinese','china\'s','china’s'] #в стоп список включаем слова, которые являются перекрестными в разных темах
lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stop_words, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in words if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return words#' '.join(words) #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста

In [64]:
#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in body:
    prepared_texts.append(prepare_text(item))

In [65]:
# Create the Inputs of LDA model: Dictionary and Corpus

dct = corpora.Dictionary(prepared_texts)
corpus = [dct.doc2bow(line) for line in prepared_texts]


In [ ]:
# Load trained model and update the model on new data

lda_model = LdaMulticore.load(datapath("C:/Users/Egran/YandexDisk/Work/news_project/lda_model"))
lda_model.update(corpus)
lda_model.save('lda_model')

In [53]:
# #Train new LDA model

# lda_model = LdaMulticore(corpus=corpus,
#                          id2word=dct,
#                          random_state=100,
#                          num_topics=15,
#                          passes=10,
#                          chunksize=1000,
#                          batch=False,
#                          alpha='asymmetric',
#                          decay=0.5,
#                          offset=64,
#                          eta=None,
#                          eval_every=0,
#                          iterations=100,
#                          gamma_threshold=0.001,
#                          per_word_topics=True)

# # save the model
# lda_model.save('lda_model')

In [40]:
# See the topics
lda_model.print_topics(num_topics=10)


[(14,
  '0.001*"headcount:" + 0.001*"headquarters:" + 0.000*"tongdun" + 0.000*"start-up" + 0.000*"technology" + 0.000*"u.s." + 0.000*"trade" + 0.000*"company" + 0.000*"autonomous" + 0.000*"start-ups"'),
 (13,
  '0.002*"airport" + 0.001*"daxing" + 0.001*"flight" + 0.001*"passenger" + 0.001*"airline" + 0.001*"terminal" + 0.000*"beijing" + 0.000*"image" + 0.000*"tenant" + 0.000*"aviation"'),
 (12,
  '0.005*"said" + 0.004*"trade" + 0.004*"also" + 0.004*"u.s." + 0.003*"would" + 0.003*"year" + 0.003*"could" + 0.003*"beijing" + 0.003*"company" + 0.002*"said."'),
 (11,
  '0.000*"said" + 0.000*"navy" + 0.000*"trade" + 0.000*"business" + 0.000*"said." + 0.000*"could" + 0.000*"u.s." + 0.000*"hong" + 0.000*"yuan" + 0.000*"also"'),
 (10,
  '0.000*"biden" + 0.000*"hunter" + 0.000*"said" + 0.000*"xiaoguang," + 0.000*"depiction" + 0.000*"studio" + 0.000*"china," + 0.000*"yat-sen," + 0.000*"hang" + 0.000*"artist"'),
 (4,
  '0.001*"informed" + 0.001*"u.s." + 0.001*"johnson" + 0.001*"trade" + 0.000*"spot

In [ ]:
import flair

In [ ]:
https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c

In [ ]:
#Выгружаем все свежие заголовки про Китай
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('https://www.reuters.com/places/china', headers={'Cache-Control': 'no-cache'})
#r1 = requests.get('https://www.reuters.com/search/news?blob=metals')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('h2', class_='FeedItemHeadline_headline')

#Выводим заголовки
for i in range(0, len(coverpage_news_r)):
    print(coverpage_news_r[i].get_text())
    print('///')

In [ ]:
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

In [ ]:
#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))


In [ ]:
#задаем ключевые слова по темам
topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR','trade']
topics_metals = ['metal', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']
print(summarize(prepared_texts[0], word_count=60))


In [ ]:
# отбор новостей по ключевым словам
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_econ:
            counter += 1
    if counter > 2:
        print('Economic story %s', i)
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1


### Блок металлов

In [ ]:
#Выгружаем все свежие заголовки про металлы из рейтерс
# s = requests.session()
# s.cookies.clear()

#r1 = requests.get('https://www.reuters.com/places/china', headers={'Cache-Control': 'no-cache'})
#https://www.reuters.com/news/archive/metals-news
r1 = requests.get('https://www.reuters.com/search/news?blob=Metals&sortBy=date&dateRange=all')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('h3', class_='search-result-title')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(coverpage_news_r[i].get_text())
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = 'https://www.reuters.com'+coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 3 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item


In [ ]:
#Парсинг статей про металлы из архива
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('https://www.reuters.com/news/archive/metals-news')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('div', class_='story-content')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(re.sub('\t','',coverpage_news_r[i].get_text()))
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = 'https://www.reuters.com'+coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 1 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item


In [ ]:
#Парсинг статей про металлы с сайта Kitco
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('http://www.kitcometals.com/news/')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('td', class_='text')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(re.sub('\t','',coverpage_news_r[i].get_text()))    #таким образом убираем тупые пробелы перед заглавием
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('article', itemprop='articleBody')
    try:
        x = body[0].find_all('p')
    except:
        pass

    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 1 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item
